## Imports
---

In [1]:
import nltk
nltk.download('movie_reviews')
from nltk.corpus import movie_reviews as mr
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.wsd import lesk
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package movie_reviews to
[nltk_data]     C:\Users\albert\AppData\Roaming\nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\albert\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\albert\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\albert\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\albert\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\albert\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already u

In [2]:
import sklearn
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import CountVectorizer

import pandas as pd

import matplotlib.pyplot as plt



import re #Regexs
import os #Acces a fitxers

c:\Users\Albert\Anaconda3\envs\myenv\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


## Exploració de dades
---

In [3]:
# Obtener las categorías disponibles
categories = mr.categories()
print(categories)

# Obtener las IDs de las revisiones de películas
file_ids = mr.fileids()
print(file_ids[8])



['neg', 'pos']
neg/cv008_29326.txt


## Preprocessing
---

In [4]:
def preprocessing(text):
    
    stop_words = set(stopwords.words('english'))
    
    # Tokenización
    text = re.sub(r'[^\w\s]', '', text) # Elimina signos de puntuación
    text = re.sub(r'\d+', '', text) # Elimina números
    text = text.lower() # Convierte a minúsculas
    text = re.sub(r'[\s\n]+', ' ', text) # Elimina espacios en blanco sobrantes excepto \n
    text = re.sub(r' +', ' ', text) # Elimina espacios en blanco largos
    text = re.sub(r'_', '', text) # Elimina barras bajas

    tokens = nltk.word_tokenize(text)
    
    # Eliminar stop words
    tokens = [token for token in tokens if token not in stop_words]
    
    return ' '.join(tokens)

## Train-Test division
---

In [5]:
#Creem la nostra X i la nostra y amb les nostres dades 

X, y = [], []

# Preprocesamiento de todas las reviews
for elem in file_ids:
    X.append((mr.raw(elem)))
    y.append(mr.categories(elem)[0])


In [6]:
#dividimos en train y test
X_train, X_test, y_train, y_test = train_test_split(X, y_num, test_size=0.2, random_state=69)

NameError: name 'y_num' is not defined

In [ ]:
y_num = []

for elem in y:
    if elem == 'neg':
        elem = 0
    else:
        elem = 1
    y_num.append(elem)

## Representació
---

- Lematitzar 
- Vectoritzar (fit en el train, transform en el train i en el test) (counts >= 5)
- Modelos


### Lematització i vectorització
---

In [ ]:
wnl = nltk.stem.WordNetLemmatizer()

def lemmatize(p):
  d = {'NN': 'n', 'NNS': 'n', 
       'JJ': 'a', 'JJR': 'a', 'JJS': 'a', 
       'VB': 'v', 'VBD': 'v', 'VBG': 'v', 'VBN': 'v', 'VBP': 'v', 'VBZ': 'v', 
       'RB': 'r', 'RBR': 'r', 'RBS': 'r'}
  if p[1] in d:
    return wnl.lemmatize(p[0], pos=d[p[1]])
  return p[0]

def lem_func(text):
    tokens = nltk.word_tokenize(text)
    tokens = nltk.pos_tag(tokens)
    lemmatized_tokens = [lemmatize(token) for token in tokens]
    return ' '.join(lemmatized_tokens)

In [ ]:
X_train_lemmatized = []

for text in X_train:
    X_train_lemmatized.append(lem_func(text))

In [ ]:
cv = CountVectorizer(min_df=5)
cv.fit(X_train_lemmatized)

def vect(text):
    a = cv.transform(text)
    return pd.DataFrame(a.toarray(), columns=cv.get_feature_names_out())


In [ ]:
X_train_cv = vect(X_train_lemmatized)

In [ ]:
X_train_cv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1600 entries, 0 to 1599
Columns: 9627 entries, aaron to zwicks
dtypes: int64(9627)
memory usage: 117.5 MB


### Models
---

## PART 2
---

In [ ]:
def get_synset(word, context):
    # Tokenize the context
    context_tokens = word_tokenize(context)
    
    # Use the Lesk algorithm to get the most likely synset
    synset = lesk(context_tokens, word)
    
    return synset

def get_context(word, text):
    # Divide the text into lines
    lines = text.split('\n')
    
    # Find the line that contains the word
    for line in lines:
        if word in line:
            return line

In [ ]:
for text in X:
    # Tokenize the text
    tokens = word_tokenize(text)
    
    # Initialize an empty list to hold the synsets for this text
    text_synsets = []
    
    # Iterate over the words in the text
    for word in tokens:
        # Get the context for the word
        context = get_context(word, text)
        
        # Get the synset for the word
        synset = get_synset(word, context)
        
        # Add the synset to the list of synsets for this text
        text_synsets.append(synset)
    
    # Convert the list of synsets to a tuple and add it to the list of tuples
    texts_words.append(tuple(text_synsets))
        
        
        

    


NameError: name 'X' is not defined